In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import col

In [0]:
%load_ext autoreload
%autoreload 2
# Enables autoreload; learn more at https://docs.databricks.com/en/files/workspace-modules.html#autoreload-for-python-modules
# To disable autoreload; run %autoreload 0

In [0]:
import sys

In [0]:
sys.path.append("/Workspace/Users/adpranav05@gmail.com/ecommerce-lakehouse-project/Resources") 

In [0]:
from config import get_config

In [0]:
cfg = get_config("dev")

## Setup Bronze tables

In [0]:
def auto_loader_ingest_bronze(source_path,dest_table,checkPoint_location):
  df_reader=(
    spark.readStream\
         .format('cloudFiles')
         .option("cloudFiles.format","csv")
         .option("cloudFiles.inferColumnTypes","true")
         .option("cloudFiles.schemaLocation",checkPoint_location)
         .option("header","true")
         .load(source_path)
  )
  
  transformed_df=df_reader\
    .withColumn("ingestion_timeStamp",current_timestamp())\
    .withColumn("source_file",col("_metadata.file_path"))

  transformed_df.writeStream\
               .format("delta")\
               .option("checkpointLocation",checkPoint_location)\
               .outputMode("append")\
               .trigger(availableNow=True)\
               .queryName(f"bronze_ingestion {dest_table}")\
               .toTable(dest_table)


### ingesting orders table

In [0]:
source_path=cfg["PATH_ORDERS"]

dest_table="ecommerce_dev.bronze.orders_raw"

checkPoint_location = "abfss://managed@databrickspractice1.dfs.core.windows.net/checkpoints/orders_raw"

In [0]:
auto_loader_ingest_bronze(source_path,dest_table,checkPoint_location)

### ingesting customers table

In [0]:
source_path=cfg["PATH_CUSTOMERS"]

dest_table="ecommerce_dev.bronze.customers_raw"

checkPoint_location = "abfss://managed@databrickspractice1.dfs.core.windows.net/checkpoints/customers_raw"

In [0]:
auto_loader_ingest_bronze(source_path,dest_table,checkPoint_location)

### ingesting order_items table

In [0]:
source_path_oi=cfg["PATH_ORDER_ITEMS"]

dest_table_oi="ecommerce_dev.bronze.order_items_raw"

checkPoint_location_oi= "abfss://managed@databrickspractice1.dfs.core.windows.net/checkpoints/order_items_raw"

In [0]:
auto_loader_ingest_bronze(source_path_oi,dest_table_oi,checkPoint_location_oi)

### ingesting product_review table

In [0]:
source_path_pr=cfg["PATH_REVIEWS"]

dest_table_pr="ecommerce_dev.bronze.reviews_raw"

checkPoint_location_pr= "abfss://managed@databrickspractice1.dfs.core.windows.net/checkpoints/reviews_raw"

In [0]:
auto_loader_ingest_bronze(source_path_pr,dest_table_pr,checkPoint_location_pr)

###ingesting product table

In [0]:
source_path_product=cfg["PATH_PRODUCTS"]

dest_table_product="ecommerce_dev.bronze.product_raw"

checkPoint_location_product= "abfss://managed@databrickspractice1.dfs.core.windows.net/checkpoints/product_raw"

In [0]:
auto_loader_ingest_bronze(source_path_product,dest_table_product,checkPoint_location_product)